In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Atherosclerosis"
cohort = "GSE109048"

# Input paths
in_trait_dir = "../../input/GEO/Atherosclerosis"
in_cohort_dir = "../../input/GEO/Atherosclerosis/GSE109048"

# Output paths
out_data_file = "../../output/preprocess/Atherosclerosis/GSE109048.csv"
out_gene_data_file = "../../output/preprocess/Atherosclerosis/gene_data/GSE109048.csv"
out_clinical_data_file = "../../output/preprocess/Atherosclerosis/clinical_data/GSE109048.csv"
json_path = "../../output/preprocess/Atherosclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Platelet gene expression profiling of acute myocardial infarction"
!Series_summary	"Acute myocardial infarction (AMI) is primarily due to coronary atherosclerotic plaque rupture and subsequent thrombus formation. Platelets play a key role in the genesis and progression of both atherosclerosis and thrombosis. Since platelets are anuclear cells that inherit their mRNA from megakaryocyte precursors and maintain it unchanged during their life span, gene expression (GE) profiling at the time of an AMI provides information concerning the platelet GE  preceding the coronary  event. In ST-segment elevation myocardial infarction (STEMI), a gene-by-gene analysis of the platelet GE identified five differentially expressed genes (DEGs): FKBP5, S100P, SAMSN1, CLEC4E and S100A12. The logistic regression model used to combine the GE in a STEMI vs healthy donors score showed an AUC of 0.95. The same five DEGs were externally validated using platelet GE data from 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# First, let's determine if the dataset contains gene expression data
is_gene_available = True  # Based on the background information, this dataset contains platelet gene expression data

# Analyze the sample characteristics dictionary to find the trait, age, and gender information

# For trait (Atherosclerosis)
# From the dictionary, row 1 contains 'diagnosis' which includes info about coronary artery disease
# sCAD = stable Coronary Artery Disease, which is a form of atherosclerosis
trait_row = 1  # The diagnosis information is in row 1

def convert_trait(value):
    """Convert diagnosis value to binary trait value for Atherosclerosis"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary (1 = has atherosclerosis, 0 = does not have atherosclerosis)
    if value.lower() == 'scad':  # stable Coronary Artery Disease
        return 1
    elif value.lower() == 'stemi':  # ST-elevation myocardial infarction, which involves atherosclerosis
        return 1
    elif value.lower() == 'healthy':
        return 0
    else:
        return None

# For age and gender
# The sample characteristics dictionary doesn't contain explicit information about age or gender
age_row = None  # Age information is not available
gender_row = None  # Gender information is not available

def convert_age(value):
    """Convert age value to continuous"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if value in ['male', 'm', '1', 'man']:
        return 1
    elif value in ['female', 'f', '0', 'woman']:
        return 0
    else:
        return None

# Check if trait data is available
is_trait_available = trait_row is not None

# Save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract clinical features if trait_row is not None
if trait_row is not None:
    # We need clinical_data for this step
    # For the purpose of this task, let's assume clinical_data is available from earlier steps
    try:
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,  
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the clinical features
        preview = preview_df(clinical_features)
        print(f"Clinical features preview: {preview}")
        
        # Save the clinical features to a CSV file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to: {out_clinical_data_file}")
    except NameError:
        print("Cannot extract clinical features: clinical_data not found")


Clinical features preview: {'GSM2928447': [1.0], 'GSM2928448': [1.0], 'GSM2928449': [1.0], 'GSM2928450': [1.0], 'GSM2928451': [1.0], 'GSM2928452': [1.0], 'GSM2928453': [1.0], 'GSM2928454': [1.0], 'GSM2928455': [1.0], 'GSM2928456': [1.0], 'GSM2928457': [1.0], 'GSM2928458': [1.0], 'GSM2928459': [1.0], 'GSM2928460': [1.0], 'GSM2928461': [1.0], 'GSM2928462': [1.0], 'GSM2928463': [1.0], 'GSM2928464': [1.0], 'GSM2928465': [1.0], 'GSM2928466': [0.0], 'GSM2928467': [0.0], 'GSM2928468': [0.0], 'GSM2928469': [0.0], 'GSM2928470': [0.0], 'GSM2928471': [0.0], 'GSM2928472': [0.0], 'GSM2928473': [0.0], 'GSM2928474': [0.0], 'GSM2928475': [0.0], 'GSM2928476': [0.0], 'GSM2928477': [0.0], 'GSM2928478': [0.0], 'GSM2928479': [0.0], 'GSM2928480': [0.0], 'GSM2928481': [0.0], 'GSM2928482': [0.0], 'GSM2928483': [0.0], 'GSM2928484': [0.0], 'GSM2928485': [1.0], 'GSM2928486': [1.0], 'GSM2928487': [1.0], 'GSM2928488': [1.0], 'GSM2928489': [1.0], 'GSM2928490': [1.0], 'GSM2928491': [1.0], 'GSM2928492': [1.0], 'GSM29

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Atherosclerosis/GSE109048/GSE109048_series_matrix.txt.gz


Gene data shape: (70523, 57)
First 20 gene/probe identifiers:
Index(['2824546_st', '2824549_st', '2824551_st', '2824554_st', '2827992_st',
       '2827995_st', '2827996_st', '2828010_st', '2828012_st', '2835442_st',
       '2835447_st', '2835453_st', '2835456_st', '2835459_st', '2835461_st',
       '2839509_st', '2839511_st', '2839513_st', '2839515_st', '2839517_st'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the first 20 rows shows that they are in the format "XXXXXXX_st"
# These are not standard human gene symbols, but rather probe IDs from an Affymetrix microarray
# These identifiers need to be mapped to standard human gene symbols for analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Look more closely at columns that might contain gene information
print("\nExamining potential gene mapping columns:")
potential_gene_columns = ['gene_assignment', 'mrna_assignment', 'swissprot', 'unigene']
for col in potential_gene_columns:
    if col in gene_annotation.columns:
        print(f"\nSample values from '{col}' column:")
        print(gene_annotation[col].head(3).tolist())



Gene annotation preview:
Columns in gene annotation: ['ID', 'probeset_id', 'seqname', 'strand', 'start', 'stop', 'total_probes', 'gene_assignment', 'mrna_assignment', 'swissprot', 'unigene', 'category', 'locus type', 'notes', 'SPOT_ID']
{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '29554', '69091', '160446', '317811'], 'stop': ['14409', '31109', '70008', '161525', '328581'], 'total_probes': [49.0, 60.0, 30.0, 30.0, 191.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His)

### Step 6: Gene Identifier Mapping

In [7]:
# Based on the gene identifiers and gene annotation data, we need to map probe IDs to gene symbols

# Examine the gene identifiers in gene_data (probe IDs look like XXXXXXX_st)
print("First few gene identifiers in gene_data:")
print(gene_data.index[:5])

# Looking at the gene annotation columns, we need to:
# 1. Find the column with probe IDs that match gene_data index format
# 2. Find the column with gene symbols for mapping

# Check which column in gene_annotation contains probe IDs matching gene_data
# The "ID" column in gene_annotation is not in the same format as gene_data.index
# Need to see what index values actually exist in gene_data to make correct mapping

# First, check if ID values actually exist in the gene_data index
print("\nChecking matching probe IDs in gene_data:")
sample_ids = gene_data.index[:5].tolist()
print(f"Sample IDs from gene_data: {sample_ids}")

# Looking for a better match between gene_data.index and the gene_annotation columns
# From the output, we need to determine which column contains the probe IDs that match gene_data index

# Since the data appears to be from an Affymetrix HTA 2.0 array (based on the '_st' suffix),
# we need to find the right probe ID column and gene symbol column

# Infer that 'ID' is the appropriate probe column, and 'gene_assignment' contains gene symbols
# We'll need to use extract_human_gene_symbols to parse the gene_assignment field

# Get the gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'gene_assignment')
print("\nGene mapping sample (first 5 rows):")
print(gene_mapping.head())

# Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print("\nMapped gene expression data shape:", gene_data.shape)
print("First few gene symbols after mapping:")
print(gene_data.index[:10])

# Normalize gene symbols to handle synonyms
gene_data = normalize_gene_symbols_in_index(gene_data)
print("\nGene data shape after normalization:", gene_data.shape)

# Save the gene expression data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")


First few gene identifiers in gene_data:
Index(['2824546_st', '2824549_st', '2824551_st', '2824554_st', '2827992_st'], dtype='object', name='ID')

Checking matching probe IDs in gene_data:
Sample IDs from gene_data: ['2824546_st', '2824549_st', '2824551_st', '2824554_st', '2827992_st']

Gene mapping sample (first 5 rows):
                ID                                               Gene
0  TC01000001.hg.1  NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-As...
1  TC01000002.hg.1  ENST00000408384 // MIR1302-11 // microRNA 1302...
2  TC01000003.hg.1  NM_001005484 // OR4F5 // olfactory receptor, f...
3  TC01000004.hg.1  OTTHUMT00000007169 // OTTHUMG00000002525 // NU...
4  TC01000005.hg.1  NR_028322 // LOC100132287 // uncharacterized L...



Mapped gene expression data shape: (71528, 57)
First few gene symbols after mapping:
Index(['A-', 'A-2', 'A-52', 'A-575C2', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V',
       'A0'],
      dtype='object', name='Gene')

Gene data shape after normalization: (24018, 57)


Gene expression data saved to: ../../output/preprocess/Atherosclerosis/gene_data/GSE109048.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols using NCBI database
print("Normalizing gene symbols...")
gene_data = pd.read_csv(out_gene_data_file, index_col=0)
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")
print("First 10 normalized gene symbols:")
print(gene_data.index[:10])

# Save the normalized gene data
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to: {out_gene_data_file}")

# 2. Extract and prepare clinical data from the matrix file
print("\nPreparing clinical data...")

# Get the clinical data rows
_, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
_, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Define convert_trait function to ensure it's available in this step
def convert_trait(value):
    """Convert diagnosis value to binary trait value for Atherosclerosis"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary (1 = has atherosclerosis, 0 = does not have atherosclerosis)
    if value.lower() == 'scad':  # stable Coronary Artery Disease
        return 1
    elif value.lower() == 'stemi':  # ST-elevation myocardial infarction, which involves atherosclerosis
        return 1
    elif value.lower() == 'healthy':
        return 0
    else:
        return None

# Process clinical data using the correct parameters from Step 2
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=1,  # Correct trait_row from Step 2
    convert_trait=convert_trait,
    age_row=None,
    convert_age=None,
    gender_row=None,
    convert_gender=None
)

print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to: {out_clinical_data_file}")

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
if linked_data.shape[0] > 0 and linked_data.shape[1] > 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data)

# 4. Handle missing values
print("\nHandling missing values...")
linked_data_clean = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 5. Check for bias in the dataset
print("\nChecking for bias in dataset features...")
is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait)

# 6. Conduct final quality validation
note = "This GSE109048 dataset contains platelet gene expression data from ST-segment elevation myocardial infarction (STEMI) patients, stable Coronary Artery Disease (sCAD) patients, and healthy donors, relevant to atherosclerosis."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data_clean,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file, index=True)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associative studies. Linked data not saved.")

Normalizing gene symbols...
Gene data shape after normalization: (24018, 57)
First 10 normalized gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2ML1-AS1',
       'A2ML1-AS2', 'A2MP1', 'A4GALT'],
      dtype='object', name='Gene')


Normalized gene data saved to: ../../output/preprocess/Atherosclerosis/gene_data/GSE109048.csv

Preparing clinical data...
Clinical data preview:
{'GSM2928447': [1.0], 'GSM2928448': [1.0], 'GSM2928449': [1.0], 'GSM2928450': [1.0], 'GSM2928451': [1.0], 'GSM2928452': [1.0], 'GSM2928453': [1.0], 'GSM2928454': [1.0], 'GSM2928455': [1.0], 'GSM2928456': [1.0], 'GSM2928457': [1.0], 'GSM2928458': [1.0], 'GSM2928459': [1.0], 'GSM2928460': [1.0], 'GSM2928461': [1.0], 'GSM2928462': [1.0], 'GSM2928463': [1.0], 'GSM2928464': [1.0], 'GSM2928465': [1.0], 'GSM2928466': [0.0], 'GSM2928467': [0.0], 'GSM2928468': [0.0], 'GSM2928469': [0.0], 'GSM2928470': [0.0], 'GSM2928471': [0.0], 'GSM2928472': [0.0], 'GSM2928473': [0.0], 'GSM2928474': [0.0], 'GSM2928475': [0.0], 'GSM2928476': [0.0], 'GSM2928477': [0.0], 'GSM2928478': [0.0], 'GSM2928479': [0.0], 'GSM2928480': [0.0], 'GSM2928481': [0.0], 'GSM2928482': [0.0], 'GSM2928483': [0.0], 'GSM2928484': [0.0], 'GSM2928485': [1.0], 'GSM2928486': [1.0], 'GSM2928487':

Linked data shape: (57, 24019)
Linked data preview (first 5 rows, 5 columns):
            Atherosclerosis      A1BG  A1BG-AS1      A1CF       A2M
GSM2928447              1.0  6.471813  2.250198  0.743049  4.202583
GSM2928448              1.0  6.336136  2.166351  0.774551  4.305025
GSM2928449              1.0  6.359611  2.162430  0.809359  4.136589
GSM2928450              1.0  6.689616  2.110528  0.881079  4.223449
GSM2928451              1.0  6.672700  2.214330  0.805369  4.152748

Handling missing values...


Linked data shape after handling missing values: (57, 24019)

Checking for bias in dataset features...
For the feature 'Atherosclerosis', the least common label is '0.0' with 19 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Atherosclerosis' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Atherosclerosis/cohort_info.json


Linked data saved to ../../output/preprocess/Atherosclerosis/GSE109048.csv
